In [19]:
tf.reset_default_graph()

In [20]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint

pp = pprint.PrettyPrinter(indent=4)

In [21]:
sess = tf.InteractiveSession()

h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [22]:

with tf.variable_scope('one_cell') as scope:
    hidden_size = 5
    cell = rnn.BasicRNNCell(num_units=hidden_size)
    print(cell.output_size, cell.state_size)
    
    x_data = np.array([[h]], dtype=np.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

5 5
array([[[-0.23859112,  0.16691491, -0.03353529, -0.52239597,
         -0.23799817]]], dtype=float32)


In [58]:
with tf.variable_scope("two_sequences") as scope:
    hidden_size = 2
    cell = rnn.BasicRNNCell(num_units= hidden_size)
    x_data = np.array([[h,e,l,l,o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[-0.64600235, -0.46835384],
        [ 0.22800814, -0.47548133],
        [-0.74445033,  0.36695424],
        [-0.44042757,  0.8370173 ],
        [-0.46592006,  0.21520786]]], dtype=float32)


In [23]:
with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2 # state is tuple 합치지않고 그대로 내보낸다 
                    # ([1,2],[3,4]), ([1,2,3,4])
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
array([[[ 0.08703703,  0.04585733],
        [-0.01946421,  0.04445234],
        [ 0.09440693, -0.02584014],
        [ 0.18173976, -0.08364522],
        [ 0.15552482, -0.11607003]],

       [[-0.09363785,  0.00303701],
        [-0.03325496, -0.08431406],
        [ 0.07316379, -0.15945068],
        [ 0.16367324, -0.17807034],
        [ 0.23212491, -0.191696

In [1]:
import tensorflow as tf
import numpy as np
import pprint

num_classes = 5
hidden_size = 5
input_dim = 5 # one_hot size
batch_size = 1 # output from LSTM, 5 to directly predict one-hot
sequence_length = 6

idx2char = ['h','i','e','l','o']
#            0   1   2   3   4
x_data = [[0, 1, 0, 2, 3, 3]]    # hihell

x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [[1, 0, 2, 3, 3, 4]]    # ihello

X = tf.placeholder(tf.float32, shape= [None, sequence_length, input_dim])
Y = tf.placeholder(tf.float32, shape= [None, sequence_length])

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _state = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
# fc_w = tf.get_variable("fc_w", [hidden_size, num_classes])
# fc_b = tf.get_variable("fc_b", [num_classes])
# outputs = tf.matmul(X_for_fc, fc_w) + fc_b
outputs = tf.contrib.layers.fully_connected(inputs=X_for_fc,
                                    num_outputs=num_classes, activation_fn=None)

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=logits, target=Y, weights=weights)

cost = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


TypeError: ('Keyword argument not understood:', 'state_is_tuple')

In [12]:
import tensorflow as tf
import numpy as np
import pprint

hidden_size = 5
input_dim = 5
batch_size = 1
sequence_length = 6

idx2char = ['h', 'i', 'e', 'l', 'o']
#            0    1    2    3    4

x_data = [[0,1,0,2,3,3]] # hihell

x_one_hot = [[[1, 0, 0, 0, 0],       # h 0
              [0, 1, 0, 0, 0],       # i 1
              [1, 0, 0, 0, 0],       # h 0
              [0, 0, 1, 0, 0],       # e 2
              [0, 0, 0, 1, 0],       # l 3
              [0, 0, 0, 1, 0]]]      # l 3

y_data = [[1,0,2,3,3,4]]

X = tf.placeholder(tf.float32, shape=[None, sequence_length, input_dim])
Y = tf.placeholder(tf.int32, shape= [None, sequence_length])

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _state = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y,
                                                weights=weights)
cost = tf.reduce_mean(sequence_loss)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

prediction = tf.argmax(outputs, axis=2)
sess = tf.InteractiveSession()

with tf.variable_scope("cell") as scope:
    sess.run(tf.global_variables_initializer())
    
    for i in range(2001):
        l, _ = sess.run([cost, optimizer], feed_dict ={X:x_one_hot, Y:y_data})
        
        result = sess.run(prediction, feed_dict={X:x_one_hot})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data, end='\t')
        result_str = [idx2char[c] for c in np.squeeze(result)]
 
        print("\tPrediction str: ", ''.join(result_str))
        
        

0 loss: 1.6216993 prediction:  [[0 0 0 0 0 1]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  hhhhhi
1 loss: 1.6123463 prediction:  [[0 0 0 0 0 1]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  hhhhhi
2 loss: 1.6035563 prediction:  [[0 3 0 0 0 1]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  hlhhhi
3 loss: 1.595235 prediction:  [[0 3 0 0 0 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  hlhhho
4 loss: 1.5872693 prediction:  [[0 3 0 0 0 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  hlhhho
5 loss: 1.5795516 prediction:  [[0 3 0 0 4 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  hlhhoo
6 loss: 1.5720067 prediction:  [[0 3 0 0 4 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  hlhhoo
7 loss: 1.5645838 prediction:  [[0 3 0 3 4 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  hlhloo
8 loss: 1.5572376 prediction:  [[0 3 0 3 4 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  hlhloo
9 loss: 1.549923 prediction:  [[0 3 0 3 4 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Predic

82 loss: 0.93249106 prediction:  [[1 0 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihelll
83 loss: 0.9280464 prediction:  [[1 0 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihelll
84 loss: 0.92369556 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
85 loss: 0.9194348 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
86 loss: 0.9152599 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
87 loss: 0.9111657 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
88 loss: 0.90714616 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
89 loss: 0.90319306 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
90 loss: 0.89929706 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
91 loss: 0.8954466 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 

167 loss: 0.71361095 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
168 loss: 0.7126531 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
169 loss: 0.7117079 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
170 loss: 0.7107752 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
171 loss: 0.7098546 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
172 loss: 0.70894545 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
173 loss: 0.7080474 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
174 loss: 0.7071603 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
175 loss: 0.70628357 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
176 loss: 0.7054169 prediction:  [[1 0 2 3 3 4]] true Y:  [[1

247 loss: 0.6605919 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
248 loss: 0.6601348 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
249 loss: 0.6596827 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
250 loss: 0.6592358 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
251 loss: 0.6587939 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
252 loss: 0.65835714 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
253 loss: 0.6579258 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
254 loss: 0.6574994 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
255 loss: 0.6570783 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
256 loss: 0.65666246 prediction:  [[1 0 2 3 3 4]] true Y:  [[1,

407 loss: 0.6270011 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
408 loss: 0.62690675 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
409 loss: 0.6268131 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
410 loss: 0.62672 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
411 loss: 0.6266275 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
412 loss: 0.62653565 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
413 loss: 0.62644434 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
414 loss: 0.6263536 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
415 loss: 0.62626344 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
416 loss: 0.6261739 prediction:  [[1 0 2 3 3 4]] true Y:  [[1,

...

1591 loss: 0.60142845 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1592 loss: 0.60142225 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1593 loss: 0.60141593 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1594 loss: 0.60140973 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1595 loss: 0.6014035 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1596 loss: 0.6013973 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1597 loss: 0.6013911 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1598 loss: 0.6013848 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1599 loss: 0.6013786 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1600 loss: 0.6013725 prediction:  [[1 0 2 3 3 4]] t

1688 loss: 0.6008486 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1689 loss: 0.60084295 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1690 loss: 0.6008372 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1691 loss: 0.60083145 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1692 loss: 0.6008258 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1693 loss: 0.60082006 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1694 loss: 0.60081434 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1695 loss: 0.6008087 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1696 loss: 0.60080296 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1697 loss: 0.6007974 prediction:  [[1 0 2 3 3 4]] 

1783 loss: 0.60032713 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1784 loss: 0.6003218 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1785 loss: 0.6003166 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1786 loss: 0.6003113 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1787 loss: 0.6003061 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1788 loss: 0.60030085 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1789 loss: 0.60029554 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1790 loss: 0.60029036 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1791 loss: 0.60028505 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1792 loss: 0.6002798 prediction:  [[1 0 2 3 3 4]] 

1874 loss: 0.5998645 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1875 loss: 0.59985954 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1876 loss: 0.59985465 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1877 loss: 0.59984976 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1878 loss: 0.5998449 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1879 loss: 0.5998399 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1880 loss: 0.5998351 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1881 loss: 0.5998302 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1882 loss: 0.5998253 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1883 loss: 0.5998205 prediction:  [[1 0 2 3 3 4]] tr

1966 loss: 0.59942836 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1967 loss: 0.5994238 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1968 loss: 0.5994192 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1969 loss: 0.59941465 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1970 loss: 0.59941006 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1971 loss: 0.5994055 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1972 loss: 0.59940094 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1973 loss: 0.5993963 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1974 loss: 0.59939176 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]		Prediction str:  ihello
1975 loss: 0.5993872 prediction:  [[1 0 2 3 3 4]] 

In [37]:
import tensorflow as tf
import numpy as np

tf.set_random_seed(66)

sample = "if you want you"
idx2char = list(set(sample))
# ['a', 'o', 'u', 'y', 'f', 'n', 't', 'w', 'i', ' ']
# [ 0,   1,   2,   3,   4,   5,   6,   7,   8,   9 ]
char2idx = {c:i for i, c in enumerate(idx2char)}
# {'a': 0,'o': 1,'u': 2,'y': 3,'f': 4,'n': 5,'t': 6,'w': 7,'i': 8,' ': 9}

input_dim = len(idx2char)
hidden_size = len(idx2char) # one 길이
num_classes = len(char2idx)
batch_size = 1
sequence_length = len(sample) - 1
learning_rate = 0.1

sample_idx = [char2idx[i] for i in sample]
x_data = [sample_idx[:-1]]
# [[8, 4, 9, 3, 1, 2, 9, 7, 0, 5, 6, 9, 3, 1]]
y_data = [sample_idx[1: ]]
# [[4, 9, 3, 1, 2, 9, 7, 0, 5, 6, 9, 3, 1, 2]]
X = tf.placeholder(tf.int32, [None, sequence_length])
# if you want yo
Y = tf.placeholder(tf.int32, [None, sequence_length])
# f you want you
x_one_hot = tf.one_hot(X, num_classes) # [None, sequence_length, num_classes]
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.2877784 Prediction: ooouuuuuouuuuu
1 loss: 2.1144257 Prediction:   ouuuuu  uuuu
2 loss: 1.9300356 Prediction:   ouuuw     ou
3 loss: 1.6599116 Prediction:   youwwan  you
4 loss: 1.3391882 Prediction: y you wan  you
5 loss: 1.0342706 Prediction: y you want you
6 loss: 0.79474646 Prediction:   you want you
7 loss: 0.59021324 Prediction:   you want you
8 loss: 0.4263638 Prediction: f you want you
9 loss: 0.2888384 Prediction: f you want you
10 loss: 0.19469787 Prediction: f you want you
11 loss: 0.12883423 Prediction: f you want you
12 loss: 0.08387839 Prediction: f you want you
13 loss: 0.054364778 Prediction: f you want you
14 loss: 0.035679758 Prediction: f you want you
15 loss: 0.024078764 Prediction: f you want you
16 loss: 0.016908946 Prediction: f you want you
17 loss: 0.012414484 Prediction: f you want you
18 loss: 0.009466562 Prediction: f you want you
19 loss: 0.00740196 Prediction: f you want you
20 loss: 0.0058733323 Prediction: f you want you
21 loss: 0.004709269 Pr

In [36]:
tf.reset_default_graph()